In [10]:
#from mpl_toolkits.mplot3d import Axes3D
import graphlab as gl
import graphlab.aggregate as agg
import numpy as np
import matplotlib.pyplot as plt
import math

In [2]:
Raw_Image = gl.SFrame.read_csv('Image test 1.txt', delimiter = '	',
                                     header = True)

Finished parsing file /home/lycas/Image test 1.txt

Parsing completed. Parsed 100 lines in 0.149548 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[float,float,str,str,str,str,float,str,str,int,int,int,int,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /home/lycas/Image test 1.txt

Parsing completed. Parsed 2757 lines in 0.058124 secs.

In [ ]:
Raw_Image.head(1)

In [3]:
Sauce = gl.SFrame({'X': Raw_Image['Xc']-min(Raw_Image['Xc']),'Y': Raw_Image['Yc']-min(Raw_Image['Yc'])})

In [ ]:
Sauce.show()

In [ ]:
Sauce.head(4)

In [ ]:
Grid = np.zeros((int(1+ max(Sauce['X'])/10),int(1+ max(Sauce['Y'])/10)))

In [ ]:
Sauce_Len = Sauce.num_rows()
for count in range (0,Sauce_Len):
    X_val = int(Sauce['X'][count]/10)
    Y_val = int(Sauce['Y'][count]/10)
    Grid[X_val][Y_val] =  Grid[X_val][Y_val] +1

In [ ]:
Grid

In [4]:
Cluster = gl.dbscan.create(Sauce, radius = 30, min_core_neighbors = 20)

Starting brute force nearest neighbors model training.

Starting blockwise similarity graph construction.

max rows per data block: 8190

number of reference data blocks: 2

+---------+-------------+--------------+

...

+-----------------------------+

| 1787                        |

| 0                           |

+-----------------------------+

[INFO] graphlab.toolkits.clustering.dbscan: Processing boundary points.


In [ ]:
Cluster.summary()

In [ ]:
Cluster['cluster_id'].print_rows(5)

In [5]:
Sauce = Sauce.add_row_number('row_id')
Sauce = Sauce.join(Cluster['cluster_id'], on='row_id', how='left')
Sauce = Sauce.rename({'cluster_id': 'dbscan_id'})

In [6]:
core_mask = Sauce['type'] == 'core'
boundary_mask = Sauce['type'] == 'boundary'
noise_mask = Sauce['type'] == 'noise'



In [7]:
#Color by cluster
plt.style.use('ggplot')
plt.axis([0, 5000, 0, 4000])
plt.scatter(Sauce['X'][core_mask],Sauce['Y'][core_mask],s=10,c=Sauce['dbscan_id'][core_mask],alpha = 0.9)
plt.scatter(Sauce['X'][boundary_mask],Sauce['Y'][boundary_mask],s=10,c=Sauce['dbscan_id'][boundary_mask],alpha = 0.5)
plt.scatter(Sauce['X'][noise_mask],Sauce['Y'][noise_mask],s=10,c='black',alpha = 0.3)
plt.show()

In [ ]:
#RAW
plt.style.use('ggplot')
plt.axis([0, 5000, 0, 4000])
plt.scatter(Sauce['X'][core_mask],Sauce['Y'][core_mask],s=20)#,c=Sauce['dbscan_id'][core_mask],alpha = 0.9)
plt.scatter(Sauce['X'][boundary_mask],Sauce['Y'][boundary_mask],s=20)#,c=Sauce['dbscan_id'][boundary_mask],alpha = 0.5)
plt.scatter(Sauce['X'][noise_mask],Sauce['Y'][noise_mask],s=20)#,c='black',alpha = 0.3)
plt.show()

In [8]:
Cluster_Size = Sauce.groupby(key_columns='dbscan_id',operations = {'center_X' : agg.MEAN('X'),'center_Y': agg.MEAN('Y')})

In [9]:
Size = Sauce.join(Cluster_Size,on = 'dbscan_id', how = 'left')

In [14]:
Size.add_column(((Size['X']-Size['center_X'])**2+(Size['Y']-Size['center_Y'])**2)**0.5, name = 'Distance')

row_id,X,Y,dbscan_id,type,center_X,center_Y,Distance
1503,2324.1,1124.6,20,core,2351.86153846,1146.22857143,35.1923019962
1466,3248.1,512.4,4,core,3273.84642857,509.18125,25.9468482816
1424,3372.3,244.8,9,core,3376.57297297,224.605405405,20.6417041173
1421,1989.5,1431.9,2,core,1983.08780488,1439.8804878,10.2374035715
1401,1737.5,1788.2,0,core,1735.8,1777.4,10.9329776365
1379,3095.1,655.4,1,core,3092.77,679.065714286,23.7801373556
10,1982.0,1448.9,2,core,1983.08780488,1439.8804878,9.08487312463
2628,2138.0,1722.7,3,core,2129.67241379,1713.9,12.1156383255
2270,1972.3,1460.1,2,core,1983.08780488,1439.8804878,22.9173603955
615,1977.8,1441.6,2,core,1983.08780488,1439.8804878,5.56035993596


In [41]:
Size_Measure = Size.groupby(key_columns='dbscan_id',operations={'stdev_size':agg.STDV('Distance')} )

In [17]:
Rad = Size.add_column((Size['X']-Size['center_X'])/Size['Distance'],name = 'Angle')

In [19]:
Rad['in Rad']= Rad['Angle'].apply(lambda x:math.acos(x))

In [34]:
Rad['Angle Group']=Rad['in Rad'].apply(lambda x:(x/math.pi * 6))

In [35]:
Area = Rad.groupby(key_columns=['dbscan_id','Angle Group'],operations={'Max Dist':agg.MAX('Distance')})

In [39]:
Num_points = Size.groupby(key_columns='dbscan_id',operations={'Num_points':agg.COUNT('row_id')})

In [41]:
Num_points

dbscan_id,Num_points
13,57
14,140
20,91
None,1282
12,42
22,31
10,37
19,64
24,50
2,41
